In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd "/content/drive/MyDrive/CI/"
!pip install import-ipynb

## Imports

In [ ]:
import import_ipynb
import utility
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# check hardware acceleration
device_name = tf.test.gpu_device_name()
print('GPU: ', device_name)

importing Jupyter notebook from utility.ipynb
Tensorflow Version: 2.4.1
GPU:  /device:GPU:0


In [ ]:
data_path='/content/drive/MyDrive/CI/numpy_data/'

In [ ]:
BATCH_SIZE = 32
IMAGE_HEIGHT = 150
IMAGE_WIDTH = 150

NORMALIZATION_VALUE = 65536

## Preprocessing

### Data-Set Building

In [ ]:
images, labels = utility.load_training()

train_abnormal_imgs, train_abnormal_labels = utility.mass_calcification_training_set_building(images, labels)

print('Shape:',np.shape(train_abnormal_imgs))
print('Training Labels:', train_abnormal_labels)

(2676, 150, 150)
Shape: (2676, 150, 150)
Training Labels: [0 0 0 ... 1 1 1]


Build the test set:

In [ ]:
test_images, test_labels = utility.load_test()

test_abnormal_imgs, test_abnormal_labels = utility.mass_calficication_test_set_building(test_images, test_labels)

### Data-Set Overview

In [ ]:
plt.hist(train_abnormal_labels)
plt.show()
mass_samples = len([label for label in train_abnormal_labels if label == 0])
print('Mass Samples:', mass_samples)
calcification_samples = len([label for label in train_abnormal_labels if label == 1])
print('Calcification Samples:',calcification_samples)

In [ ]:
train_samples, labels = utility.oversample(train_abnormal_imgs, train_abnormal_labels)

Oversampling must be done on 240 elements of the first array.
Class 0: 1458
Class 1: 1458


In [ ]:
plt.hist(labels, range=(0,1))
plt.show()
mass_samples = len([label for label in labels if label == 0])
print('Mass Samples:', mass_samples)
calcification_samples = len([label for label in labels if label == 1])
print('Calcification Samples:',calcification_samples)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

train_samples_reshaped = np.expand_dims(train_samples,3)

test_samples_reshaped = np.expand_dims(test_abnormal_imgs,3)

val_split = 0.2
X_train, X_val, y_train, y_val = train_test_split(train_samples_reshaped, labels, test_size=val_split, stratify=labels)


train_datagen = ImageDataGenerator(rescale=1./NORMALIZATION_VALUE)

train_generator = train_datagen.flow(X_train, 
                                     y=y_train, 
                                     batch_size=BATCH_SIZE, 
                                     seed=123)
validation_generator = train_datagen.flow(X_val, 
                                     y=y_val, 
                                     batch_size=BATCH_SIZE, 
                                     seed=123)

print('Training and Validation generated.')

test_datagen = ImageDataGenerator(rescale=1./NORMALIZATION_VALUE)

test_generator =  test_datagen.flow(test_samples_reshaped,
                                y=test_abnormal_labels,
                                batch_size=BATCH_SIZE,
                                seed=123)

print('Test Generated')

## Training

## V2

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

model_v2 = models.Sequential(name='v2')
model_v2.add(tf.keras.Input(shape=(150,150,1)))

model_v2.add(layers.experimental.preprocessing.RandomFlip(mode='horizontal', seed=123)) #paper 
model_v2.add(layers.experimental.preprocessing.RandomRotation(factor=0.25, seed=123, fill_mode='nearest')) #90degree

model_v2.add(layers.Conv2D(32, (3, 3), activation='relu'))
model_v2.add(layers.MaxPooling2D((2, 2)))
model_v2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_v2.add(layers.MaxPooling2D((2, 2)))
model_v2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_v2.add(layers.MaxPooling2D((2, 2)))
model_v2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_v2.add(layers.MaxPooling2D((2, 2)))
model_v2.add(layers.Flatten())
model_v2.add(layers.Dense(512, activation='relu'))
model_v2.add(layers.Dense(1, activation='sigmoid'))

model_v2.summary()

In [ ]:
from tensorflow.keras import optimizers

opt = keras.optimizers.RMSprop(learning_rate=0.001)

model_v2.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

EPOCHS = 35

In [ ]:
history_v2 = model_v2.fit(train_generator,
          validation_data=validation_generator,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

In [ ]:
utility.plot_history(history_v2)

In [ ]:
dir_name ="/content/drive/MyDrive/CI/models"
if not os.path.exists(dir_name):
  os.makedirs(dir_name)
model_v2.save(os.path.join(dir_name,'MC_SCRATCH.h5'))

In [ ]:
test_loss, test_accuracy = model_v2.evaluate(test_generator)

print('Test Accuracy:', test_accuracy)
print('Test Loss:', test_loss)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

normalized_test_samples_reshaped = test_samples_reshaped/NORMALIZATION_VALUE
test_predictions = model_v2.predict(normalized_test_samples_reshaped)

accuracy, confusion_matrix, auc, f2 = utility.do_evaluations(test_predictions, test_abnormal_labels)


fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()

fig.savefig('/content/drive/MyDrive/CI/plot/final/mc_scratch_conf_matrix.png')